In [1]:
import tqdm
import re
import pandas as pd
import math
import time

# Collect focal papers

In [2]:
def get_abstract(words):
    n = max({idx  for name in words for idx in words[name]})+1
    temp_list = [' ']*n
    for word in words:
        idxs = words[word]
        for idx in idxs:
            temp_list[idx] = word
    abstract = ' '.join(temp_list)
    return abstract

def get_basic_infos(doc):
    list_concepts = [concept['id'] for concept in doc['concepts']]
    Source = None
    if doc['primary_location']:
        if 'source' in doc['primary_location']:
            if doc['primary_location']['source']:
                if 'id' in doc['primary_location']['source']:
                    Source = int(re.sub("https://openalex.org/S","",doc['primary_location']['source']['id']))
    infos = {'id':re.sub("https://openalex.org/","",doc['id']),
    'doi':doc['doi'],
    'year':doc['publication_year'],
    'language':doc['language'],
    'type':doc['type'],
    'source':Source,
    'nb_auth':len(doc['authorships']),
    'auth_ids':'; '.join([re.sub("https://openalex.org/A","",auth['author']['id']) for auth in doc['authorships'] if auth['author']['id']]),
    'countries' : '; '.join([auth['countries'][0] for auth in doc['authorships'] if auth['countries']]),
    'institutions' : '; '.join([auth['institutions'][0]['id'] for auth in doc['authorships'] if auth['institutions']]),
    'nb_citations': doc['cited_by_count'],
    'title': doc['title'],
    'abstract': get_abstract(doc['abstract_inverted_index']) if doc['abstract_inverted_index'] else None,
    'nb_ref':doc['referenced_works_count'] ,
    'concepts': '; '.join(list_concepts),
    'references': '; '.join(doc['referenced_works'])}
    
    return infos

def get_nb_pages(url, limit=50):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        count = data['meta']['count']
        nb_pages = math.ceil(count/limit)
        print(f"Number of articles matched: {count}")
        print(f"Number of pages: {nb_pages}")
    return nb_pages

def get_articles(url,cursor = False,limit = 100):
    if cursor:
        url += f"&cursor={cursor}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()    
        articles = data['results']
        if cursor:
            next_cursor = data['meta']['next_cursor']
            return articles, next_cursor
        else:
            return articles
    else:
        print("Failed to retrieve data:", response.status_code)
        return None

import requests
import json

def get_url(search, time_windows, concept_id, limit=100):
    url = "https://api.openalex.org/works?"
    if search:
        search = search.replace(' ', '+')
        title_search = f"search={search}"
        url += title_search
   
    if time_windows or concept_id:
        if search:
            filter_ = '&filter='
        else:
            filter_ = 'filter='
        url += filter_
        
        if time_windows:
            filter_year = f"publication_year:{time_windows}"
            url += filter_year
        if concept_id:
            if time_windows:
                url += ','
            filter_concept = f"concepts.id:{concept_id}"
            url += filter_concept
    url += "&mailto=s.bianchini@unistra.fr"
    url += f"&per-page={limit}"
    return url

In [3]:
def get_concept_id(query):
    query = query.replace(' ', '+')
    url = f"https://api.openalex.org/concepts?search={query}&per-page=200"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        print(f"Number of concepts with {query} in display_name: {data['meta']['count']}")
        concepts = data['results']
        return concepts
    else:
        print("Failed to retrieve data:", response.status_code)
        return None
    
    
concepts = get_concept_id('labour economics')
pd.DataFrame(concepts)

Number of concepts with labour+economics in display_name: 1


,id,wikidata,display_name,relevance_score,level,description,works_count,cited_by_count,summary_stats,ids,image_url,image_thumbnail_url,international,ancestors,related_concepts,counts_by_year,works_api_url,updated_date,created_date
0,https://openalex.org/C145236788,https://www.wikidata.org/wiki/Q28161,Labour economics,116484.914,1,functioning and dynamics of the markets for la...,569577,6498058,"{'2yr_mean_citedness': 1.1874455353511881, 'h_...",{'openalex': 'https://openalex.org/C145236788'...,None,None,"{'display_name': {'ar': 'اقتصاديات العمل', 'as...","[{'id': 'https://openalex.org/C162324750', 'wi...","[{'id': 'https://openalex.org/C162324750', 'wi...","[{'year': 2024, 'works_count': 8457, 'cited_by...",https://api.openalex.org/works?filter=concepts...,2024-09-01T12:17:38.151583,2016-06-24


In [ ]:
concepts = [#"C140096630",
#"C3020597237",
#"C2780451532",
#"C96250715",
#"C15952604",
#"C187691185",
"C185798385",
"C2777526511",
"C132010649",
"C18483071",
"C170477896",
"C66283442",
"C46362747",
"C2779679103",
"C74172769",
"C94487597",
"C2779356329",
"C2779918689",
"C51399673",
"C87360688",
"C138171918",
"C143916079",
"C52121051",
"C2779134260",
"C121608353",
"C104317684",
"C204787440",
"C116675565",
"C118518473",
"C2777267654"
]

import os
time_windows = '2000-2023'
search = None
limit = 200
path_ = os.getcwd()
    
for concept_id in concepts:
    print(concept_id)
    os.chdir(path_)
    if not os.path.exists(concept_id):
        os.makedirs(concept_id)
    os.chdir(concept_id)
    url = get_url(search, time_windows, concept_id, limit)
    nb_pages = get_nb_pages(url, limit)
    all_infos = []
    next_cursor = '*'
    for page in tqdm.tqdm(range(1,nb_pages+1)):
        try:
            time.sleep(1)
            page_i, next_cursor = get_articles(url,next_cursor,limit)
            for article in page_i:
                infos = get_basic_infos(article)
                all_infos.append(infos)
        except Exception as e :
            print(e)
            time.sleep(30)
            page_i, next_cursor = get_articles(url,next_cursor,limit)
            for article in page_i:
                infos = get_basic_infos(article)
                all_infos.append(infos)
            continue
          
    df = pd.DataFrame(all_infos)
    df.to_csv('Data_concept_{}.csv'.format(concept_id))
    
    works_ids = set()
    for refs in df.references.to_list():
        refs = re.sub("https://openalex.org/","",refs)
        refs = refs.split("; ")
        works_ids.update(refs)
    
    path = "mongodb://"
    import pymongo
    client = pymongo.MongoClient(path)
    db = client['Stefano']
    col = db[concept_id]
    
    
    all_infos = []
    works_ids = list(works_ids)[1:]
    for i in tqdm.tqdm(range(0,len(works_ids),100)):
        to = i+100 if i+100<len(works_ids) else len(works_ids)
        work_ids = "|".join(works_ids[i:to])
        url = "https://api.openalex.org/works?filter=ids.openalex:{}&per-page=100".format(work_ids)
        try:
            time.sleep(1)
            page_i = get_articles(url)
            for article in page_i:
                infos = get_basic_infos(article)
                col.insert_one(infos)
        except Exception as e :
            print(e)
            time.sleep(1000)
            page_i = get_articles(url)
            for article in page_i:
                infos = get_basic_infos(article)            
                col.insert_one(infos)
            continue
    
    pd.DataFrame(all_infos).to_csv('Data_ref_concept_{}.csv'.format(concept_id))
    
    df = pd.read_csv('Data_concept_{}.csv'.format(concept_id))
    #df_ref = pd.read_csv('Data_ref_concept_{}.csv'.format(concept_id))
    df_ref = pd.DataFrame([doc for doc in col.find()])
    
    df["references"] = df['references'].str.split('; ')
    
    edgelist_ref = df[["id","year","references"]].explode('references')
    edgelist_ref["references"] = edgelist_ref["references"].apply(lambda x:  re.sub("https://openalex.org/","",x) if isinstance(x,str) else None)
    
    ref_infos = df_ref[["id","source"]]
    ref_infos = ref_infos.rename(columns={'id': 'references'})
    
    merge_infos = edgelist_ref.merge(ref_infos, on = 'references', how = "left")
    
    merge_infos = merge_infos.dropna()
    
    ref_structured = merge_infos.groupby(["id","year"]).apply(lambda x: [ref for ref in x["source"]]).reset_index()
    
    ref_structured = ref_structured.rename(columns={0: 'refs'})
    
    ref_structured['id'] = ref_structured['id'].apply(lambda x: int(re.sub('W','',x)))
    if not os.path.exists("Data/docs/refs/"):
        os.makedirs("Data/docs/refs/")
    for y in range(2000,2024):
        tmp = ref_structured[ref_structured.year == y]
        json.dump(tmp.to_dict('records'),open("Data/docs/refs/"+ "{}.json".format(y),'w'))

C185798385
Number of articles matched: 232151
Number of pages: 1161


100%|███████████████████████████████████████████████████████████████████████████▉| 1160/1161 [1:00:27<00:03,  3.07s/it]

'countries'


 49%|███████████████████████████████████▋                                     | 9285/19018 [7:40:54<7:34:09,  2.80s/it]

Failed to retrieve data: 503
'NoneType' object is not iterable


...

100%|████████████████████████████████████████████████████████████████████████████| 4386/4386 [3:31:26<00:00,  2.89s/it]


C52121051
Number of articles matched: 68007
Number of pages: 341


100%|████████████████████████████████████████████████████████████████████████████| 2663/2663 [2:03:51<00:00,  2.79s/it]


C2779134260
Number of articles matched: 3583465
Number of pages: 17918


 71%|██████████████████████████████████████████████████▍                    | 12744/17918 [11:35:28<3:51:20,  2.68s/it]